# Contacts between SARS-CoV-2 and ACE2

To better understand the complex between the RBD and ACE2, we did some analysis of contacts between RBD and ACE2. Using the Bio.PDB Python library, I was able to find all contacts between the SARS-CoV-2 RBD and ACE2 using the 6M0J structure. For our purposes, if a residue C Alpha atom is within 7 Angstroms of another residue of C Alpha atom, the two residues are considered in contact. Results showed that there are 16 contacts.

In [3]:
from tabulate import tabulate
import numpy as np
from Bio.PDB import *

In [4]:
parser = PDBParser(PERMISSIVE=True, QUIET=True)
data = parser.get_structure("6moj", "6m0j.pdb")
model = data.get_models()

models = list(model)
chains = list(models[0].get_chains())
residuesA = list(chains[0].get_residues())
residuesE = list(chains[1].get_residues())
# residue identifier, atom
cAlphaA = [[], []]
cAlphaE = [[], []]
for i in range(len(residuesA)):
    atoms = list(residuesA[i].get_atoms())
    for j in range(len(atoms)):
        if atoms[j].get_id() == "CA":
            cAlphaA[0].append(residuesA[i].get_resname() + str(residuesA[i].get_id()[1]))
            cAlphaA[1].append(atoms[j])
            break
for i in range(len(residuesE)):
    atoms = list(residuesE[i].get_atoms())
    for j in range(len(atoms)):
        if atoms[j].get_id() == "CA":
            cAlphaE[0].append(residuesE[i].get_resname() + str(residuesE[i].get_id()[1]))
            cAlphaE[1].append(atoms[j])
            break

# RBD residue, count contacts with ACE2, list of contacted residues
data = [[], [], []]

countTot = 0
for i in range(len(cAlphaE[0])):
    count = 0
    contacts = ""
    for j in range(len(cAlphaA[0])):
        if(cAlphaE[1][i] - cAlphaA[1][j]) <= 7:
            count += 1
            countTot += 1
            contacts += cAlphaA[0][j] + ", "
    contacts = contacts[:len(contacts)-2]
    if count > 0:
        data[0].append(cAlphaE[0][i])
        data[1].append(count)
        data[2].append(contacts)

aceRes = set()
for i in range(len(data[2])):
    res = data[2][i].split(", ")
    for s in res:
        aceRes.add(s)

dataInRows = []
for i in range(len(data[0])):
    arr = [data[0][i], data[1][i], data[2][i]]
    dataInRows.append(arr)

In [5]:
print("Total Contacts: " + str(countTot))
print("Cutoff Distance: 7 Angstroms")
print()
print(str(len(data[0])) + " RBD Contact Residues:", end=" ")
for c in data[0]:
    print(c, end=" ")
print()
print(str(len(aceRes)) + " ACE2 Contact Residues:", end = " ")
for c in aceRes:
    print(c, end=" ")
print()
print()
print(tabulate((dataInRows), headers=["RBD Residue", "Number of Contacts with ACE2", "ACE2 Residues"]))

Total Contacts: 16
Cutoff Distance: 7 Angstroms

8 RBD Contact Residues: ALA475 GLY476 GLY496 THR500 ASN501 GLY502 GLY504 TYR505 
7 ACE2 Contact Residues: ASP355 SER19 GLY354 LYS353 GLY326 ASP38 GLN24 

RBD Residue      Number of Contacts with ACE2  ACE2 Residues
-------------  ------------------------------  ------------------------------
ALA475                                      1  GLN24
GLY476                                      2  SER19, GLN24
GLY496                                      1  ASP38
THR500                                      2  GLY326, ASP355
ASN501                                      3  LYS353, GLY354, ASP355
GLY502                                      4  GLY326, LYS353, GLY354, ASP355
GLY504                                      1  GLY354
TYR505                                      2  LYS353, GLY354
